# Universal differential equations

Esse notebook é uma demonstração de 'Universal differential equations' (UDE) usando modelos de circuitos equivalentes e PyBaMM. Uma UDE é uma equação diferencial definito, por todo ou em parte, em função de "aproximadores universais" (redes-neurais nesse caso).

Na prática, são sistema físicos, descritos por equações diferenciais, onde um dos componentes é uma rede neural[1]. Exemplo: $$\frac{\partial u}{\partial t} = f(x,u,t) + \mathcal{NN}(x,u)$$

Vantagem dessa implementação: $f(\cdot)$ pode ser um modelo físico simplificado (*reduced order*) que retorna uma mera aproximação do resultado real. Nesse caso, a rede neural $\mathcal{NN}$ pode ser menor, mais simples, responsável somente por corrigir o erro, a divergência.

---
- Rackauckas et al 2021 (https://doi.org/10.48550/arXiv.2001.04385).

## Exemplo Neural-TECMD
O melhor exemplo, com código open-source disponível online, vem de Kuzhiyil et al, 2024 [2, 3]. O software é implementado em Julia, consegui faze-lo rodar localmente, mas não no Google Colab. Vou demonstrar abaixo a lógica básica do programa e o output do código -- não será possível executar localmente agora.

O artigo descreve o processo de combinar modelo  *Thermal Equivalent Circuit Model with Diffusion* (TECMD) -- um dos modelos do tipo ECM mais sofisticados -- com duas redes neurais que corrigem o erro de temperatura e tensão. Na figura abaixo, blocos pretos representam modelos físicos-matemáticos, blocos laranjas representam redes neurais.

![test](https://github.com/SarvTex/june_sciml_workshop/blob/main/imgs/neural_tecmd_kuzhiyil.jpg?raw=1)

O modelo conseguiu obter reduções de erro significativo, enquanto preserva performance.

A figura abaixo demonstra a comparação entre dados experimentais, e os resultados da simulação com e sem redes-neurais auxiliares.

![test](https://github.com/SarvTex/june_sciml_workshop/blob/main/imgs/neural_tecmd_plot.png?raw=1){width:300px}

---
- Kuzhiyil et al 2024 (https://doi.org/10.1016/j.apenergy.2024.123692)
- https://github.com/JishnuKuzhiyil/Neural-TECMD-Battery-model

## Inicialização Colab
Caso esse notebook esteja sendo executado pelo Google Colab, algumas modificações tem que ser feitas. O bloco abaixo reorganiza esses aspectos.

In [1]:
# importing basic libs
import os
import sys

# setting up code to run in google colab
git_repo_url = "https://github.com/SarvTex/june_sciml_workshop.git"
repo_name = "june_sciml_workshop"

if 'google.colab' in sys.modules:
    # Google colab setup
    print("> Code running in Colab env ===")

    if not repo_name in os.getcwd().split(os.sep) and not repo_name in os.listdir():
        print("> Cloning repo (dataset) ===")
        !git clone {git_repo_url}
    else:
      print("> Repo already cloned ===")

    if repo_name in os.listdir():
        os.chdir(repo_name)
    print(f"> Currently in correct folder ({os.getcwd()}) ===")

else:
    # Running locally setup
    print("> Running locally. Assuming correct folder structure ===")

!ls

> Code running in Colab env ===
> Cloning repo (dataset) ===
Cloning into 'june_sciml_workshop'...
remote: Enumerating objects: 448, done.
remote: Counting objects: 100% (448/448), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 448 (delta 18), reused 439 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (448/448), 37.76 MiB | 10.29 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Updating files: 100% (398/398), done.
> Currently in correct folder (/content/june_sciml_workshop) ===
01_pinnModel.ipynb		   data		       imgs
02_udeModel.ipynb		   enviroment.yml      README.md
03_physicsFeature_synthData.ipynb  explore_data.ipynb  test.ipynb


## RNN para ODE
Abaixo uma representação básica de uma RNN sendo usada para resolver um problema de equações diferenciais.  
Esse exemplo vem de Nascimento et al (2020), código disponível em Github. Esse artigo usa formação de rupturas mecânicas baseado na [Lei de Paris Erdogan](https://en.wikipedia.org/wiki/Paris'_law).

Aqui, a rede neural RNN opera como uma '*integradora*', ou seja, computa a derivada no tempo de $y_n$ e determina um novo valor para $y_{n+1}$. Nesse exemplo em específico, a integração é baseado no alg de Euler, uma das metodologias de integrações mais simples.

![img](imgs/typesOfCell_nascimentoRNN.jpg)

---
- "[Nascimento, R. G., Fricke, K., & Viana, F. A. C. (2020)](https://www.sciencedirect.com/science/article/pii/S095219762030292X). A tutorial on solving ordinary differential equations using Python and hybrid physics-informed neural networks. Engineering Applications of Artificial Intelligence, 96, 103996."

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#from tensorflow.keras.layers import RNN, Dense, Layer
#from tensorflow.keras import Sequential
#from tensorflow.keras.optimizers import RMSprop
#from tensorflow.python.framework import tensor_shape
#from tensorflow import float32, concat, convert_to_tensor

: 

No tensorflow, pode-se criar "*layers*" customizadas. Para usá-las em um RNN, programa-se esse layer como uma "*cell*". obrigatório definir as funções `__init__` e `call` dessa classe. Exemplo de pseudo-código abaixo.
```python
# pseudo-code ----
class CustomCell(tf.keras.layers.Layer):
    def __init__(self, x): # ...
    def call(self, x): # ...

rnn_cell = CustomCell(x)
rnn = tf.keras.layers.RNN(cell=rnn_cell, ...)
# ---
```

Dentro da `EulerIntergratorCell`, existe uma rede neural `dKlayer`, que calcula o termo $\Delta K$ da equação de Paris.

In [ ]:
class EulerIntegratorCell(tf.keras.layers.Layer):
    def __init__(self, C, m, dKlayer, a0=None, units=1, **kwargs):
        super(EulerIntegratorCell, self).__init__(**kwargs)
        self.units = units
        self.C     = C
        self.m     = m
        self.a0    = a0
        self.dKlayer     = dKlayer
        self.state_size  = tf.TensorShape(self.units)
        self.output_size = tf.TensorShape(self.units)

    def build(self, input_shape, **kwargs):
        self.built = True

    def call(self, inputs, states):
        inputs  = tf.convert_to_tensor(inputs)
        a_tm1   = tf.convert_to_tensor(states)
        x_d_tm1 = tf.concat((inputs, a_tm1[0, :]), axis=1)
        dk_t    = self.dKlayer(x_d_tm1)
        da_t    = self.C * (dk_t ** self.m)
        a       = da_t + a_tm1[0, :]
        return a, [a]

    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        return self.a0

Outro uso para a função "layer customizada": Camada de 'normalização'.

In [ ]:
class Normalization(tf.keras.layers.Layer):
    def __init__(self, S_low, S_up, a_low, a_up, **kwargs):
        super(Normalization, self).__init__(**kwargs)
        self.low_bound_S   = S_low
        self.upper_bound_S = S_up
        self.low_bound_a   = a_low
        self.upper_bound_a = a_up

    def build(self, input_shape, **kwargs):
        self.built = True

    def call(self, inputs):
        output  = (inputs - [self.low_bound_S, self.low_bound_a]) / [(self.upper_bound_S - self.low_bound_S), (self.upper_bound_a - self.low_bound_a)]
        return output

In [ ]:

def create_model(C, m, a0, dKlayer, batch_input_shape, return_sequences=False, return_state=False):
    euler = EulerIntegratorCell(C=C, m=m, dKlayer=dKlayer, a0=a0, batch_input_shape=batch_input_shape)
    PINN  = tf.keras.layers.RNN(cell=euler, batch_input_shape=batch_input_shape, return_sequences=return_sequences, return_state=return_state)
    model = tf.keras.Sequential()
    model.add(PINN)
    model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(1e-2))
    return model

Dentro da 'Euler cell', existe uma rede neural que preve $\Delta K$ chamada `dKlayer`. Esse rede é pré-iniciada aqui, antes mesmo de se criar o RNN. Nota-se que esse NN é inicialmente treinada de acordo com a fórmula `dK_range`

In [ ]:
# Paris law coefficients
[C, m] = [1.5E-11, 3.8]

# data
Strain = np.asarray(pd.read_csv('./data/ude_nascimento/Strain.csv'))[:,:,np.newaxis]
atrain = np.asarray(pd.read_csv('./data/ude_nascimento/atrain.csv'))
a0     = np.asarray(pd.read_csv('./data/ude_nascimento/a0.csv'))[0,0]*np.ones((Strain.shape[0],1))

# stress-intensity layer
dKlayer = tf.keras.Sequential()
dKlayer.add(Normalization(np.min(Strain), np.max(Strain), np.min(atrain), np.max(atrain)))
dKlayer.add(tf.keras.layers.Dense(5, activation='tanh'))
dKlayer.add(tf.keras.layers.Dense(1))

# weight initialization
S_range  = np.linspace(np.min(Strain), np.max(Strain), 1000)
a_range  = np.linspace(np.min(atrain), np.max(atrain), 1000)[np.random.permutation(np.arange(1000))]
dK_range = -12.05 + 0.24 * S_range + 760.0 * a_range

dKlayer.compile(loss='mse', optimizer=tf.keras.optimizer.RMSprop(1e-2))
inputs_train = np.transpose(np.asarray([S_range, a_range]))
dKlayer.fit(inputs_train, dK_range, epochs=100)

In [ ]:

# fitting physics-informed neural network
model = create_model(C=C, m=m, a0=tf.convert_to_tensor(a0, dtype=tf.float32), dKlayer=dKlayer, batch_input_shape=Strain.shape)
aPred_before = model.predict_on_batch(Strain)[:,:]
model.fit(Strain, atrain, epochs=100, steps_per_epoch=1, verbose=1)
aPred = model.predict_on_batch(Strain)[:,:]

# plotting predictions
fig = plt.figure()
plt.plot([0,0.05],[0,0.05],'--k')
plt.plot(atrain, aPred_before, 'o', label = 'before training')
plt.plot(atrain, aPred, 's', label = 'after training')
plt.xlabel("actual crack length (m)")
plt.ylabel("predicted crack length (m)")
plt.legend(loc = 'upper center',facecolor = 'w')
plt.grid(which='both')
plt.show()

## Modelo próprio
Abaixo um modelo próprio baseado no que foi apresentado acima.  
Essa rede-neural (RNN) representa um ECM básico que é composto de uma fonte de tensão e uma resitência R0.

O modelo simplificado da rede de tensão segue a simplificação: $ V(x) = U_0 + \beta\cdot\log(\frac{x}{1-x}) $.

Acho que essa tentativa tem potencial, ainda vou continuar trabalhando nisso.

In [36]:
class SimpleEquivalentCircuitModel(tf.keras.layers.Layer):
    """
    These are the model inputs:
        U_max | Max voltage
        U_min | Min voltage
        Ro | Internal resistance Ro
        ecm_nn | The neural net for correcting the physical model (pre-trained)
        soc_0 = 0.95, | Initial state of charge
        time_step = 10, | Time step
        units = 3, | Number of outputs/inner states
    """

    def __init__(self,
                 U_max:np.float32,
                 U_min:np.float32,
                 Ro:np.float32,
                 capacity:np.float32,
                 ecm_nn,
                 soc_0 = 0.95,
                 time_step = 10,
                 units = 2,
                 **kwargs):
        super(SimpleEquivalentCircuitModel, self).__init__(**kwargs)
        self.U0 = 0.5 * (U_max + U_min)
        self.dU = 0.5 * (U_max - U_min) * 0.2175
        self.Ro = Ro
        self.capacity = capacity
        self.soc_0 = soc_0
        self.dt = time_step
        self.ecm_nnet = ecm_nn
        # units
        self.state_size = tf.TensorShape(units)
        self.output_size = tf.TensorShape(units)

    def build(self, input_shape, **kwargs):
        self.built = True

    def v_out_phys(self, x_t):
        # x_t: inputs & state concat
        # x_t features: (curr, temp, v_out, soc)
        return self.U0 + self.dU * tf.math.log(x_t[:,3]/(1-x_t[:,3])) - self.Ro * x_t[:,0]

    def soc_phys(self, x_t):
        # coulomb counting:
        # dSoc/dt = -I/3600/C
        # x_t features: (curr, temp, v_out, soc)
        return x_t[:,3] - x_t[:,0] * self.dt / (3600 * self.capacity)


    def call(self, inputs, states):
        # input_t and output_{t-1}
        inputs = tf.convert_to_tensor(inputs) # should be a (x,2) tensor: [0] = current, [1] = temperature
        out_m1 = tf.convert_to_tensor(states) # should be a (x,2) tensor: [0] = v_out, [1] = soc
        x_t = tf.concat((inputs, out_m1[0,:]), axis=1)

        out = tf.stack([
            self.v_ocv_phys(x_t),
            self.soc_phys(x_t)
        ])
        out = out - self.ecm_nnet(x_t)
        return out, [out]


#### Explain

```python
    def call(self, inputs, states):
        # input_t and output_{t-1}
        inputs = tf.convert_to_tensor(inputs) # should be a (x,2) tensor: [0] = current, [1] = temperature
        out_m1 = tf.convert_to_tensor(states) # should be a (x,3) tensor: [0] = v_out, [1] = soc, [2] = v_rc
        x_t = tf.concat((inputs, out_m1[0,:]), axis=1)
```
Note: input should be a (x,2) tensor and states should be a list
which gets converted to a (1,x,3) tensor by the convert_to_tensor
method, hence this line

### Usando PyBaMM

Python Battery Mathematical Modelling (PyBaMM) é um pacote de Python open-source para simulação eletroquímica de baterias -- tipicamente a classe de modelos mais sofisticada e completa. Essa é uma ferramenta potencialmente muito útil dado nosso interesse em avaliação e diagnóstico de baterias por meio de modelos de simulação.

Durante essa demonstração inicial, vamos comparar um modelo ECM básico com o modelo eletroquímico DFN do PyBaMM.